## Introduction
Because of the unique scoring of American Football, where scoring most commonly occurs in intervals of 7 or 3 points (and more rarely, 2, 6, or 8), the final scores of NFL games tend to take on certain discrete values; some are incredibly common and occur thousands of times while others have yet to occur. NFL fans on the internet have tracked the occurance of NFL final scores and fans have given the name "scoirgami" to the event of a novel scoring combination occuring for the first time.

The below project attempts various machine learning techniques for predicting if an in-progress NFL game will end in a scorigami.

## Outline
1. Importing Libraries and dataset
2. Establishing baselines for prediction
3. Random Forest algorithm
4. Deep Neural Network
5. Pretraining DNN
5. Recurrant Neural Network

## 1. Importing Libraries and Dataset

In [1]:

# Load libraries
import numpy as np
from sklearn.model_selection import train_test_split

import math
from sklearn.metrics import precision_recall_curve,auc,average_precision_score
import matplotlib.pyplot as plt
import tensorflow as tf
import gc

from tensorflow.keras import layers, regularizers
from sklearn.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import PrecisionRecallDisplay

X is in the shape (6757, 60, 16). This corresponds to 6,473 NFL games occuring between the 2001 and 2025 season. In each game, 60 observations are made corresponding to each minute of gameplay in a regulation football game (not including overtime) of 16 features of interest corresponding to the cumulative number of different scoring plays that have occured in that game. The feature labels are contained in the file X_features.csv

y is an array of booleans representing whether each of the games would have been a scorigami had it occured in the 2001 season.

In [63]:
# Load dataset
# See getData.py for details on how dataset was obtained and cleaned

X = np.load('X.npy')
y = np.load('y.npy')
print(X.shape)
print(y.shape)

with open('X_features.csv', 'r') as f:
  print(f.read())

#Split into train/test/cross-validation

X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2)

(6757, 60, 16)
(6757, 1)
homeTds, awayTds, homePATs, awayPATs, home2Cvs, away2Cvs, homeFGs, awayFGs, homeSafeties, awaySafeties, totalSafeties, homeDef2ptConv, awayDef2ptConv, currentHome, currentAway, totalPoints, 


Features are recorded as "home" and "away" teams. However, for purposes of Scorigami, it does not matter if the teams are home or away; therefore, the X_train dataset is duplicated, reversing "home" and "away" teams.

In [64]:
#Experimental: Because the dataset is split into "home" and "away", this function duplicates the training set with swapping the home and away team data


X_train_r = np.ndarray(X_train.shape, float)

X_train_r[:, :, 0] = X_train[:, :, 1]
X_train_r[:, :, 1] = X_train[:, :, 0]
X_train_r[:, :, 2] = X_train[:, :, 3]
X_train_r[:, :, 3] = X_train[:, :, 2]
X_train_r[:, :, 4] = X_train[:, :, 5]
X_train_r[:, :, 5] = X_train[:, :, 4]
X_train_r[:, :, 6] = X_train[:, :, 7]
X_train_r[:, :, 7] = X_train[:, :, 6]
X_train_r[:, :, 8] = X_train[:, :, 9]
X_train_r[:, :, 9] = X_train[:, :, 8]
X_train_r[:, :, 10] = X_train[:, :, 10]
X_train_r[:, :, 11] = X_train[:, :, 12]
X_train_r[:, :, 12] = X_train[:, :, 11]
X_train_r[:, :, 13] = X_train[:, :, 14]
X_train_r[:, :, 14] = X_train[:, :, 13]
X_train_r[:, :, 15] = X_train[:, :, 15]

X_train = np.append(X_train, X_train_r, axis=0)
print(X_train.shape)
y_train = np.append(y_train, y_train, axis=0)
print(y_train.shape)


(10810, 60, 16)
(10810, 1)


To demonstrate predicting an in-progress NFL game, a "time snapshot" will be made at the 45th minute of the game. The X array will be a 2D array containing only observations of the interest features at the 45th minute of the game

In [68]:
time = 45
X_train_snapshot = X_train[:,time-1,:]
X_test_snapshot = X_test[:,time-1,:]

print(X_train_snapshot.shape)
print(X_test_snapshot.shape)

(10810, 16)
(1352, 16)


# 2. Baseline Models

To evaluate the performance of my machine learning models, I will create two different deterministic predictors as baseline comparisons. The first one predicts that a game will end in a scorigami if the current score

In [77]:
# import isScorigami function used in Current Scorigami Predictor
!wget https://github.com/liam-moyer/Scorigami-Modeling/raw/refs/heads/main/EveryScorigami.xlsx
!pip install fastexcel
from isScorigami import isScorigami

#Current Scorigami predictor: if a scorigami at time t, then predict scorigami
def currentscorigamiredictor(testSet):
  out = []
  for i in range(testSet.shape[0]):
    if isScorigami(testSet[i,'currentHome'],testSet[i,'currentAway'],2001):
      out.append(1)
    else:
      out.append(0)
  return out


meanTotalScore = np.average(X_train_snapshot[:,15])
stdTotalScore = np.std(X_train_snapshot[:,15])

  #Predict positive if home + away is more than 2 SD from mean
def scorepredictor(testSet):
  out = []
  for i in range(testSet.shape[0]):
    if testSet[i,15] > (meanTotalScore + 2*stdTotalScore):
      out.append(1)
    else:
      out.append(0)
  return out


--2026-02-05 02:04:10--  https://github.com/liam-moyer/Scorigami-Modeling/raw/refs/heads/main/EveryScorigami.xlsx
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/liam-moyer/Scorigami-Modeling/refs/heads/main/EveryScorigami.xlsx [following]
--2026-02-05 02:04:10--  https://raw.githubusercontent.com/liam-moyer/Scorigami-Modeling/refs/heads/main/EveryScorigami.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1326019 (1.3M) [application/octet-stream]
Saving to: ‘EveryScorigami.xlsx.6’

EveryScorigami.xlsx 100%[===================>]   1.26M  --.-KB/s    in 0.06s   

2026-02-05 02:04:10 (22.0 MB/s

In [ ]:
# Train model using tuned hyperparamters

best_hps = tuner.get_best_hyperparameters(5)
for i in range(5):
  print(str(best_hps[i].values))

model = hypermodel.build(best_hps[0])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="f1_score",mode='max',patience=30, restore_best_weights=True) # callbacks=[early_stopping_cb]
hypermodel.fit(hp,model,x_train_np, np.array(y_train).reshape(-1, 1), epochs=100, callbacks=[early_stopping_cb])

predicted_ANN = model.predict(x_test_np)

In [66]:
from tensorflow.keras.layers import LSTM, Dense

model = tf.keras.Sequential([
    LSTM(100, activation='tanh', return_sequences=True, input_shape=(X.shape[1],X.shape[2])),  # First LSTM layer
    LSTM(100, activation='tanh', return_sequences=True),  # Second LSTM layer
    LSTM(100, activation='tanh', return_sequences=True),  # Second LSTM layer
    LSTM(100, activation='tanh', return_sequences=True),  # Second LSTM layer
    LSTM(100, activation='tanh'),
    Dense(100, activation="relu"),
    Dense(100, activation="relu"),
    Dense(100, activation="relu"),
    Dense(100, activation="relu"),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

loss = tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True,alpha=0.9,gamma=2)
model.compile(loss=loss,optimizer = tf.keras.optimizers.AdamW(), metrics=[tf.keras.metrics.AUC(curve='PR'), tf.keras.metrics.F1Score(average="macro", threshold=0.5)])
model.summary()

model.fit(X_train, y_train, epochs=100, batch_size=200, validation_split=0.2)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 60, 100)        │        46,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 60, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 60, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 60, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 408,901 (1.56 MB)

 Trainable params: 408,901 (1.56 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - auc_1: 0.0461 - f1_score: 0.0206 - loss: 0.0204 - val_auc_1: 0.1154 - val_f1_score: 0.0000e+00 - val_loss: 0.0196
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc_1: 0.1200 - f1_score: 0.1679 - loss: 0.0184 - val_auc_1: 0.2120 - val_f1_score: 0.2443 - val_loss: 0.0183
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc_1: 0.2098 - f1_score: 0.2554 - loss: 0.0183 - val_auc_1: 0.2175 - val_f1_score: 0.2016 - val_loss: 0.0185
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc_1: 0.1847 - f1_score: 0.1984 - loss: 0.0175 - val_auc_1: 0.2882 - val_f1_score: 0.3485 - val_loss: 0.0168
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc_1: 0.2216 - f1_score: 0.3152 - loss: 0.0155 - val_auc_1: 0.2777 - val_f1_score: 0.2878 - val_loss: 0.0161
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - auc_1: 0.2112 - f1_score: 0.2644 - loss: 0.0157 - val_auc_1: 0.2834 - val_f1_score: 0.3420 - val_loss: 0.0155
Epoch 7/100


In [ ]:
# Creates synthetic dataset to simulate NFL scoring according to Poisson distribution

x_syn=np.ndarray((0,16),int)

for i in range(70000):
  homeTDs = np.random.poisson(lam=2.4)
  awayTDs = np.random.poisson(lam=2.4)
  homePATs = 0
  awayPATs = 0
  home2Cvs = 0
  away2Cvs = 0
  homeDef2ptConv = 0
  awayDef2ptConv = 0
  for j in range(homeTDs):
    k = np.random.uniform()
    if k <= 0.001:
      awayDef2ptConv += 1
    if k <= 0.05:
      home2Cvs +=1
    if k<= 0.95:
      homePATs += 1
  for j in range(awayTDs):
    k = np.random.uniform()
    if k <= 0.001:
      homeDef2ptConv += 1
    if k <= 0.05:
      away2Cvs += 1
    if k <= 0.95:
      awayPATs += 1
  homeFG = np.random.poisson(lam=1.6)
  awayFG = np.random.poisson(lam=1.6)
  homeSafeties = np.random.poisson(lam=0.05)
  awaySafeties = np.random.poisson(lam=0.05)
  totalSafeties = homeSafeties + awaySafeties
  currentHome = homeTDs*6 + homePATs + home2Cvs*2 + homeDef2ptConv * 2 +  homeFG*3 + homeSafeties*2
  currentAway = awayTDs*6 + awayPATs + away2Cvs*2 + awayDef2ptConv *2 + awayFG*3 + awaySafeties*2
  totalPoints = currentHome + currentAway
  x_syn = np.append(
      x_syn,
      [[homeTDs,awayTDs,homePATs,awayPATs,home2Cvs, away2Cvs,homeFG,awayFG,homeSafeties,awaySafeties,totalSafeties, homeDef2ptConv, awayDef2ptConv, currentHome, currentAway, totalPoints],
       [awayTDs,homeTDs,awayPATs,homePATs,away2Cvs, home2Cvs,awayFG,homeFG,awaySafeties,homeSafeties,totalSafeties, awayDef2ptConv, homeDef2ptConv, currentAway, currentHome, totalPoints]
       ],
      axis=0
  )


print(x_syn)

y_syn = []
for i in range(x_syn.shape[0]):
  #y_syn.append(isScorigami(x_syn[i,'currentHome'],x_syn[i,'currentAway'],2001))
  y_syn.append(isScorigami(x_syn[i,13],x_syn[i,14],2001))
print(y_synth)
fig, ax = plt.subplots()
#ax.scatter(x_syn['currentHome'],x_syn['currentAway'],c=y_syn,cmap='viridis')
ax.scatter(x_syn[:,13],x_syn[:,14],c=y_syn,cmap='viridis')
plt.grid(True)
plt.show()
print(f"Positives: {sum(y_syn)/ len(y_syn)}")

In [ ]:
# Clones base ANN model and trains on synthetic dataset

modelA = tf.keras.models.clone_model(model)
modelA.set_weights(model.get_weights())
modelA.fit(x_syn, np.array(y_syn).reshape(-1, 1), batch_size=1200,epochs=300)

model_A_clone = tf.keras.models.clone_model(modelA)
model_A_clone.set_weights(modelA.get_weights())
from tensorflow.keras import layers, regularizers

model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-3])


n_neurons = best_hps[0].get('n_neurons')
b_alpha = best_hps[0].get('alpha')
b_gamma = best_hps[0].get('gamma')
#b_dropout = best_hps[0].get('dropout')
b_lr = best_hps[0].get('learning_rate')
l2reg = best_hps[0].get("l2_regularization")

print(n_neurons)
for layer in model_B_on_A.layers:
  layer.trainable = False
model_B_on_A.add(tf.keras.layers.Dense(n_neurons, activation="leaky_relu",kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(l2=l2reg)))
model_B_on_A.add(tf.keras.layers.Dense(n_neurons, activation="leaky_relu",kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(l2=l2reg)))
model_B_on_A.add(tf.keras.layers.Dropout(rate=0.15))
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model_B_on_A.compile(loss = tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=b_alpha, gamma=b_gamma),
                      optimizer = tf.keras.optimizers.AdamW(learning_rate=b_lr),  metrics=[tf.keras.metrics.AUC(curve='PR')])

model_B_on_A.fit(x=x_train_np,
          y=np.array(y_train).reshape(-1, 1),
          epochs=100,
          batch_size=200,
          shuffle=True
)

for layer in model_B_on_A.layers:
  layer.trainable = True

optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-6)
model_B_on_A.compile(loss = tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=b_alpha, gamma=b_gamma),
                      optimizer = optimizer,  metrics=[tf.keras.metrics.AUC(curve='PR')])

model_B_on_A.fit(x=x_train_np,
          y=np.array(y_train).reshape(-1, 1),
          epochs=100,
          batch_size=200,
          shuffle=True,
          #validation_split=0.1
)

predicted_ANN_transfer = model_B_on_A.predict(x_test_np)

In [ ]:
y_true = np.array(y_test, dtype=int)
pos = y_true.sum()
n = y_true.size
prevalence = pos / n
print(f"Positives: {pos} / {n}  (prevalence = {prevalence:.4f})")

In [ ]:
# Display metrics for base ANN model

y_score = predicted_ANN.flatten().round()

precision, recall, thresholds = precision_recall_curve(y_true, y_score)

aucprANN = auc(recall, precision)
ap = average_precision_score(y_true, y_score)

print(f"AUCPR (trapezoidal) = {aucprANN:.4f}")
print(f"Average Precision(AP) = {ap:.4f}")

plt.figure(figsize=(7,5))
plt.plot(recall, precision, label=f"AUCPR = {aucprANN:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (ANN)")
plt.legend()
plt.grid(True)
plt.show()

f1ANN = f1_score(y_true,y_score)

print(f"F1 score: {f1ANN}")

disp = ConfusionMatrixDisplay.from_predictions(y_true,y_score)
plt.title("Confusion Matrix(ANN)")
plt.show()

In [ ]:
# Metrics for ANN model with transfer learning
from sklearn.metrics import ConfusionMatrixDisplay
y_score = predicted_ANN_transfer.flatten().round()

precision, recall, thresholds = precision_recall_curve(y_true, y_score)

aucprTransfer = auc(recall, precision)
ap = average_precision_score(y_true, y_score)

print(f"AUCPR (trapezoidal) = {aucprTransfer:.4f}")
print(f"Average Precision(AP) = {ap:.4f}")

plt.figure(figsize=(7,5))
plt.plot(recall, precision, label=f"AUCPR = {aucprTransfer:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (ANN with Transfer Learning)")
plt.legend()
plt.grid(True)
plt.show()

f1Transfer = f1_score(y_true,y_score)

print(f"F1 score: {f1Transfer}")
disp = ConfusionMatrixDisplay.from_predictions(y_true,y_score)
plt.title("Confusion Matrix(ANN with Transfer Learning)")
plt.show()

In [ ]:
# Metrics for "high scoring" baseline model
baseline_prediction = scorepredictor(x_test)

precision_bl,recall_bl,thresholds_bl = precision_recall_curve(y_true,baseline_prediction)
aucpr_bl_score = auc(recall_bl, precision_bl)
ap_bl = average_precision_score(y_true,baseline_prediction)

print(f"AUCPR (trapezoidal) = {aucpr_bl_score:.4f}")
print(f"Average Precision(AP) = {ap_bl:.4f}")

plt.figure(figsize=(7,5))
plt.plot(recall_bl, precision_bl, label=f"AUCPR = {aucpr_bl_score:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (high scoring baseline)")
plt.legend()
plt.grid(True)
plt.show()

f1bl_score = f1_score(y_true,baseline_prediction)

print(f"F1 score: {f1_score(y_true,baseline_prediction)}")

disp = ConfusionMatrixDisplay.from_predictions(y_true,baseline_prediction)
plt.show()

In [ ]:
# Metrics for "current scorigami" baseline model
baseline_prediction = currentscorigamipredictor(x_test)

precision_bl,recall_bl,thresholds_bl = precision_recall_curve(y_true,baseline_prediction)
aucpr_bl_naive = auc(recall_bl, precision_bl)
ap_bl = average_precision_score(y_true,baseline_prediction)

print(f"AUCPR (trapezoidal) = {aucpr_bl_naive:.4f}")
print(f"Average Precision(AP) = {ap_bl:.4f}")

plt.figure(figsize=(7,5))
plt.plot(recall_bl, precision_bl, label=f"AUCPR = {aucpr_bl_naive:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (current scorigami baseline)")
plt.legend()
plt.grid(True)
plt.show()

f1bl_naive = f1_score(y_true,baseline_prediction)

print(f"F1 score: {f1_score(y_true,baseline_prediction)}")

disp = ConfusionMatrixDisplay.from_predictions(y_true,baseline_prediction)
plt.show()

In [ ]:
# These graphs show the model outputs on 3-D plots to show the class seperation achieved by each model

fig, ax = plt.subplots()
ax.scatter(x_test['currentAway'],x_test['currentHome'],c=y_test,cmap='viridis')
plt.grid(True)
plt.show()


fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10,10))
ax.scatter(x_test['currentAway'],x_test['currentHome'],predicted_ANN.flatten(),c=y_test, linewidths=4)
plt.grid(True)
plt.show()

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10,10))
ax.scatter(x_test['currentAway'],x_test['currentHome'],predicted_ANN_transfer.flatten(),c=y_test, linewidths=4)
plt.grid(True)
plt.show()

In [ ]:
#More graphs

#density of scores
meanTotalScore = np.average(x_train.select('totalPoints').to_numpy())
stdTotalScore = np.std(x_train.select('totalPoints').to_numpy())
fig, ax = plt.subplots()
ax.hist(x_train['finalHome'] + x_train['finalAway'],bins=500)
ax.vlines(meanTotalScore,0,1,linestyle='dashed',color='red',label='Mean')
ax.vlines(meanTotalScore+stdTotalScore,0,1,linestyle='dashed',color='gray',label='1 std')
ax.vlines(meanTotalScore+ 2 * stdTotalScore,0,1,linestyle='dashed',color='gray', label="2 std")
plt.grid(True)
plt.show()

import matplotlib.pyplot as plt
from matplotlib.patches import Arc

#Classification status
meanTotalScore = np.average(x_test_np[:,15])
stdTotalScore = np.std(x_test_np[:,15])
classification = []
for i in range(x_test.shape[0]):
  if y_test[i]:
    if predicted_ANN.flatten().round()[i]:
      #'True Positive'
      classification.append(0)
    else:
      #'False Negative'
      classification.append(1)
  else:
    if predicted_ANN.flatten().round()[i]:
      #'False Positive'
      classification.append(2)
    else:
      #'True Negative'
      classification.append(3)
fig, ax = plt.subplots()
scatter = ax.scatter(x_test_np[:,13],x_test_np[:,14],c=classification,cmap='viridis',label=classification)
plt.grid(True)
plt.legend(handles=scatter.legend_elements()[0], labels=['True Positive','False Negative','False Positive', "True Neg"])
plt.show()
